In [1]:
!nvidia-smi

Fri Jan  9 11:47:10 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 590.48.01              Driver Version: 590.48.01      CUDA Version: 13.1     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...    Off |   00000000:01:00.0  On |                  N/A |
| N/A   46C    P8              5W /   60W |    1495MiB /   8188MiB |     87%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# %pip install "plotly[express]"

/home/yuki/Code/Python/pipellm/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# install numpy required by plotly.express (runs in the notebook environment)
import plotly.express as px
from collections import defaultdict

In [6]:
class CharTokenizer:
    def __init__(self, training_data: str = "the quick brown fox jumps over the lazy dog"):
        self.training_data = training_data
        self.fit()

    def fit(self):
        self.vocab = sorted(set(self.training_data))
        self.char_to_id = {char: idx for idx, char in enumerate(self.vocab)}
        self.id_to_char = {idx: char for idx, char in enumerate(self.vocab)}
        self.vocab_size = len(self.vocab)

    def plot_token_distribution(self):
        counter = defaultdict(int)
        for token in self.encode(self.training_data):
            counter[token] += 1
        freq = [counter[self.char_to_id[char]] for char in self.vocab]
        fig = px.bar(x=self.vocab, y=freq, labels={'x': 'Token', 'y': 'Frequency'}, width=800, height=300)
        fig.write_html("plot2.html", include_plotlyjs="cdn")
        fig.show()

    def get_compression_ratio(self):
        num_bytes = len(bytes(self.training_data, encoding="utf-8"))
        num_tokens = len(self.encode(self.training_data))
        return num_bytes / num_tokens

    def encode(self, text: str):
        return [self.char_to_id[char] if char in self.char_to_id else '<unk>' for char in text]

    def decode(self, ids: list):
        return ''.join([self.id_to_char[id_] if type(id_) is int else '<unk>' for id_ in ids])

tk = CharTokenizer()
print(tk.vocab_size, tk.vocab)
print(tk.encode("hi, how are you?"))
print(tk.decode(tk.encode("hi, how are you?")))
print('Compression ratio', tk.get_compression_ratio())
tk.plot_token_distribution()

with open("../data/the-verdict.txt", "r") as f:
    data = f.read()
tk = CharTokenizer(data)
print(tk.vocab_size, tk.vocab)
print(tk.encode("hi, how are you?"))
print(tk.decode(tk.encode("hi, how are you?")))
print('Compression ratio', tk.get_compression_ratio())
tk.plot_token_distribution()

27 [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
[8, 9, '<unk>', 0, 8, 15, 23, 0, 1, 18, 5, 0, 25, 15, 21, '<unk>']
hi<unk> how are you<unk>
Compression ratio 1.0


62 ['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
[43, 44, 7, 1, 43, 50, 58, 1, 36, 53, 40, 1, 60, 50, 56, 12]
hi, how are you?
Compression ratio 1.0
